# Rainfall across the Australian wheatbelt

The Australian Bureau of Agricultural and Resource Economics and Sciences (ABARES)
defines a set of Australian broadacre zones and regions.

The last four very dry years across the "wheat-sheep" region
line up really well with the last four times Australia had to import grain
(1994-95, 2002-03, 2006-07, 2019-20; see
[ABC](https://www.abc.net.au/news/rural/2019-05-15/australia-approves-grain-imports/11113320),
[Guardian](https://www.theguardian.com/australia-news/2019/may/15/australia-to-import-wheat-for-first-time-in-12-years-as-drought-eats-into-grain-production)).

In [ ]:
wheat_import_years = [1994, 2002, 2006, 2019]

In [ ]:
import os
import sys
sys.path.append('/home/599/dbi599/unseen/unseen')

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import xarray as xr
import numpy as np
from dask.distributed import Client, LocalCluster
from IPython.display import Image
import cftime

import fileio
import time_utils

In [ ]:
Image(filename='/g/data/xv83/dbi599/ag/figures/ag_regions.png')

In [ ]:
# Optional parameters 
# (This cell is tagged "parameters")
dpi = 300

In [ ]:
# Required parameters
assert 'region' in locals(), "Must provide an AGCD data file (papermill option -p region [all south-west south-east or north-east])"
assert os.path.isfile(agcd_file), "Must provide an AGCD data file (papermill option -p agcd_file [filepath])"
assert os.path.isfile(cafe_file), "Must provide an CAFE data file (papermill option -p cafe_file [filepath])"
assert os.path.isfile(cafe_bc_file), "Must provide an CAFE bias corrected data file (papermill option -p cafe_bc_file [filepath])"
assert os.path.isfile(fidelity_file), "Must provide an fidelity test file (papermill option -p fidelity_file [filepath])"
assert os.path.isfile(independence_plot), "Must provide an independence test plot (papermill option -p independence_plot [filepath])"

## Observations

In [ ]:
agcd_ds = fileio.open_file(agcd_file)

In [ ]:
agcd_ds

In [ ]:
agcd_ds['pr'] = agcd_ds['pr'] * 365
agcd_ds['pr'].attrs['units'] = 'mm yr-1'

In [ ]:
years = agcd_ds['time'].dt.year.values
agcd_df = pd.DataFrame(index=years)
agcd_df[region] = agcd_ds['pr'].sel(region=region).values

In [ ]:
mean_rainfall = agcd_df[region].mean()
print(mean_rainfall)

In [ ]:
years_list = list(years)
nyears = len(years_list)
colors = ['tab:blue'] * nyears
for year in wheat_import_years:
    index = years_list.index(year)
    colors[index] = 'tab:red'

In [ ]:
agcd_df[region].plot.bar(figsize=[20, 9], width=0.8, color=colors)
plt.axhline(mean_rainfall, color='0.5', linestyle='--')
plt.ylabel('annual precipitation (mm)')
plt.title(f'wheat-sheep ({region}) region')
plt.grid(axis='y')
plt.savefig(f'/g/data/xv83/dbi599/ag/figures/wheat_sheep_precip_{region}.png',
            bbox_inches='tight', facecolor='white', dpi=dpi)
plt.show()

In [ ]:
ranked_years = agcd_df[region].sort_values()
ranked_years.head(n=10)

In [ ]:
def year_in_context(ranked_years_df, year):
    """Put a given year in context"""
    
    nyears = len(ranked_years_df)
    rank = ranked_years_df.index.get_loc(year) + 1
    percentile = (rank / nyears) * 100
    return_period = nyears / rank
    
    print(f'# {year} statistics:')
    print(f'{rank} in {nyears} year event')
    print(f'{percentile:.1f}% percentile')
    print(f'{return_period:.0f} year return period')
    print(' ')

In [ ]:
for year in wheat_import_years:
    year_in_context(ranked_years, year)

In [ ]:
all_regions = ['all', 'south-west', 'south-east', 'north-east']
for comparison_region in all_regions:
    if not region == comparison_region:
        corr = xr.corr(agcd_ds['pr'].sel(region=region),
                       agcd_ds['pr'].sel(region=comparison_region),
                       dim='time').values
        print(f'{region} vs {comparison_region}: {corr}')

## Model ensemble

In [ ]:
cafe_ds = fileio.open_file(cafe_file)
cafe_bc_ds = fileio.open_file(cafe_bc_file)

In [ ]:
cafe_bc_ds['pr'].attrs['units']

In [ ]:
cafe_ds['pr'] = cafe_ds['pr'] * 365
cafe_ds['pr'].attrs['units'] = 'mm yr-1'

cafe_bc_ds['pr'] = cafe_bc_ds['pr'] * 365
cafe_bc_ds['pr'].attrs['units'] = 'mm yr-1'

### Bias correction and fidelity testing

In [ ]:
fig = plt.figure(figsize=[10, 6])
cafe_ds.sel(region=region)['pr'].plot.hist(bins=50, density=True, label='CAFE', alpha=0.7)
cafe_bc_ds.sel(region=region)['pr'].plot.hist(bins=50, density=True, label='CAFE BIAS CORRECTED', facecolor='darkblue', alpha=0.7)
agcd_ds.sel(region=region)['pr'].plot.hist(bins=50, density=True, label='AGCD', facecolor='green', alpha=0.7)
plt.xlabel('annual precipitation (mm)')
plt.ylabel('probability')
plt.title(f'Average precipitation across the wheat-sheep ({region}) region')
plt.legend()
plt.savefig(f'/g/data/xv83/dbi599/ag/figures/wheat_sheep_precip_histogram_{region}.png',
            bbox_inches='tight', facecolor='white', dpi=dpi)
plt.show()

In [ ]:
fidelity_ds = xr.open_zarr(fidelity_file)

In [ ]:
fidelity_ds.sel(region=region)['pval'].values

These are the p-values for each lead time.

p > 0.05 means the null hypothesis (that the two samples are from the same population) can't be rejected.

### Independence testing

In [ ]:
Image(filename=independence_plot)

### Exceedance curves

In [ ]:
cafe_bc_ds_stacked = cafe_bc_ds.sel(region=region, lead_time=slice(3, None))['pr'].stack({'sample': ['ensemble', 'init_date', 'lead_time']})

In [ ]:
cafe_bc_ds_stacked.shape

In [ ]:
cafe_bc_ds_stacked = time_utils.select_time_period(cafe_bc_ds_stacked, ['1990-01-01', '2019-12-31'])

In [ ]:
cafe_bc_ds_stacked = cafe_bc_ds_stacked.dropna(dim='sample')

In [ ]:
cafe_bc_ds_stacked.shape

In [ ]:
def calc_exceedance(ds_stacked):
    """Calculate exceedance"""
    
    data = ds_stacked.compute()
    sorted_data = np.sort(data, axis=None)
    exceedance = 1.-np.arange(1.,len(data) + 1.)/len(data)
    
    return sorted_data, exceedance

In [ ]:
sorted_data, exceedance = calc_exceedance(cafe_bc_ds_stacked)

fig = plt.figure(figsize=[8, 6])
ax = fig.add_subplot()
ax.plot(sorted_data, exceedance * 100)
ax.set_title(f'Average precipitation across the wheat-sheep ({region}) region')
ax.set_ylabel('likelihood of exceedance (%)')
ax.set_xlabel('annual precipitation (mm)')
plt.show()

In [ ]:
def event_in_context(data, threshold, year, direction):
    """Put an event in context
    
    Args:
      data (np.ndarray)
      threshold (float) : event threshold
      year (int) : year that the event occured in the observations
      direction (str) : 'less' or 'greater' than
    """

    n_population = len(data)
    if direction == 'less':
        n_events = np.sum(data < threshold)
    elif direction == 'greater':
        n_events = np.sum(data > threshold)
    else:
        raise ValueError("""direction must be 'less' or 'greater'""")
    percentile = (n_events / n_population) * 100
    return_period = 1 / (percentile / 100)
    
    print(f'# {year} ({threshold:.1f}mm) statistics:')
    print(f'{n_events} in {n_population} year event')
    print(f'{percentile:.2f}% percentile')
    print(f'{return_period:.0f} year return period')
    print(' ')

#### Full 1990-2019 time period

In [ ]:
for year in wheat_import_years:
    event_in_context(sorted_data, ranked_years[year], year, 'less')

fig = plt.figure(figsize=[8, 6])
ax = fig.add_subplot()
ax.plot(sorted_data, 100 - (exceedance * 100))
ax.invert_xaxis()
ax.set_title(f'Average precipitation across the wheat-sheep ({region}) region')
ax.set_ylabel('likelihood of deceedance (%)')
ax.set_xlabel('annual precipitation (mm)')

for year in wheat_import_years:
    ax.axvline(ranked_years[year], color='0.5', linestyle='--')

plt.savefig(f'/g/data/xv83/dbi599/ag/figures/wheat_sheep_precip_deceedence_{region}.png',
            bbox_inches='tight', facecolor='white', dpi=dpi)

#### By decade

In [ ]:
fig = plt.figure(figsize=[8, 6])
ax = fig.add_subplot()

for start_year in [1990, 1995, 2000, 2005, 2010, 2015]:
    end_year = start_year + 4
    start_date = f'{start_year}-01-01'
    end_date = f'{end_year}-12-31'
    ds_selection = time_utils.select_time_period(cafe_bc_ds_stacked, [start_date, end_date])
    ds_selection = ds_selection.dropna('sample')
    sorted_data, exceedance = calc_exceedance(ds_selection)
    n_years = len(sorted_data)
    label = f'{start_year}-{end_year} ({n_years} samples)'
    ax.plot(sorted_data, 100 - (exceedance * 100), label=label)
    
    print(f'{start_year}-{end_year}')
    for year in wheat_import_years:
        event_in_context(ds_selection.values, ranked_years[year], year, 'less')

ax.invert_xaxis()
ax.set_title(f'Average precipitation across the wheat-sheep ({region}) region')
ax.set_ylabel('likelihood of deceedance (%)')
ax.set_xlabel('annual precipitation (mm)')
ax.legend()
for year in wheat_import_years:
    ax.axvline(ranked_years[year], color='0.5', linestyle='--')

plt.savefig(f'/g/data/xv83/dbi599/ag/figures/wheat_sheep_precip_deceedence-by-pentad_{region}.png',
            bbox_inches='tight', facecolor='white', dpi=dpi)